In [ ]:
import zmq
import threading
import time
import hashlib
import random
from pow import proof_of_work
from collections import deque
#
#import logging

class StopMineException(Exception):
    pass

class Node(object):
    """docstring"""

    ctx = None
    psocket = None
    lsocket = None


    def __init__(self, ipaddr, port):
        self.ipaddr = ipaddr
        self.port = port
        self.ctx = zmq.Context.instance()
        self.psocket = self.ctx.socket(zmq.PUB)
        self.lsocket = self.ctx.socket(zmq.SUB)
        self.lsocket.setsockopt(zmq.SUBSCRIBE, b'')
        self.balance = 0
        self.stake = 0
        self.peers = deque()

    def connect(self):
        if self.peers:
            for ip in self.peers:
                self.lsocket.connect("tcp://%s:9000" % ip['ipaddr'])
        else: # no peers
            self.lsocket.connect("tcp://127.0.0.1:9000")

    def listen(self, k, e, bchain):
        while True and not k.is_set():
            try:
                block_msg = self.lsocket.recv_string()
                e.set()
                print(block_msg)
                # lock?
                bchain.appendleft(block_msg)
                e.clear()
            except (zmq.ContextTerminated):
                break

    def bind(self):
        self.psocket.bind("tcp://%s:%s" % (self.ipaddr, self.port))

    def close(self):
        self.psocket.close(linger=0)
        self.lsocket.close(linger=0)
        self.ctx.term()

    def addPeer(self, ipaddr):
        peer = {'ipaddr': ipaddr}
        if peer not in self.peers:
            self.peers.appendleft(peer)
            return True
        else:
            return False

    def removePeer(self, ipaddr):
        peer = {'ipaddr': ipaddr}
        try:
            self.peers.remove(peer)
        except ValueError:
            return False
        return True

    def getPeers(self):
        return self.peers

    def setBalance(self, value):
        self.balance = value

    def doConsensus(self, k, e, bchain):
        name = threading.current_thread().getName()
        while True and not k.is_set():
            stop = e.is_set()
            new_block = name + chr(random.randint(1,100)) + bchain[0]
            (new_hash, nonce) = proof_of_work(new_block, 21, stop)
            if new_hash != '':
                bchain.appendleft('Mined: Block'+new_hash)
                self.psocket.send_string("%s: Block %s" % (name, new_hash))

    def writeBlock(self):
        pass

    def readBlock(self):
        pass

    def checkBlock(self, e):
        return True

    def run(self, kill, e, bchain):
        m1 = threading.Thread(name='Miner',target=self.doConsensus,
         kwargs={'k': kill, 'e': e, 'bchain': bchain})
        m1.start()
        return [m1]

# Instanciação do nó e conexão

In [ ]:
bchain = deque()
bchain.appendleft('')
n = Node('127.0.0.1',9000)
n.connect()
n.bind()

# Início das threads

In [ ]:
kill = threading.Event()
e = threading.Event()

### Listen Thread

In [ ]:
listen_thread = threading.Thread(target=n.listen,
     kwargs={'k': kill, 'e': e, 'bchain': bchain})
listen_thread.start()

### Mining Thread

In [ ]:
threads = n.run(kill, e, bchain)

### Main Thread

In [ ]:
try:
    while (raw_input()) != 'exit':
        pass
except KeyboardInterrupt:
        pass
finally:
        kill.set()
        for t in threads:
            t.join()
        n.close()
        print bchain

In [1]:
!netstat -tan | grep 127.0.0.1:9000